# Analyzing Phylogeny Estimation Software on Aligned DNA Sequence Data

## Data
The estimated and true trees are in the `data/` folder of this repository. These were calculated from data described in <i>Liu et al., "Rapid and Accurate Large-Scale Coestimation of Sequence Alignments and Phylogenetic Trees," Science, vol. 324, no. 5934, pp. 1561-1564, 19 June 2009.</i> You can access the original datasets [here](https://sites.google.com/eng.ucsd.edu/datasets/alignment/sate-i?authuser=0). To view the tools used to compute the trees from this data, visit the following links:
- [FastTree](http://www.microbesonline.org/fasttree/#Install): FastTree+GTR, FastTree+JC69
- [FastME](https://gite.lirmm.fr/atgc/FastME/): NJ+LogDet, NJ+JC69, NJ+P-Distances
  
Note that FastME will require sequence alignment data to be in Phylip format and the datasets linked above will be in FASTA format. To convert the data from FASTA to Phylip, refer to the `convert.py` script in this repository.
  
## Loading Required Modules and Functions
For phylogenetic analysis, we will use dendropy along with a tree comparison function by Errin Molloy used for NJMerge. We will test out the module by calculation the false positive (FP) and negative (FN) rates between two arbitrary trees.

In [1]:
import dendropy
print(dendropy.__version__)

4.20220511.00


In [2]:
tns = dendropy.TaxonNamespace()

tree1 = dendropy.Tree.get(
    path="data/1000M1/R0/fasttree/gtrFastTree.tree", 
    schema="newick",
    taxon_namespace=tns
)
tree2 = dendropy.Tree.get(
    path="data/1000M1/R0/rose.tt", 
    schema="newick",
    taxon_namespace=tns
)

tree1.encode_bipartitions()
tree2.encode_bipartitions()

tree1.collapse_basal_bifurcation(set_as_unrooted_tree=True)
tree2.collapse_basal_bifurcation(set_as_unrooted_tree=True)

In [3]:
import compare_trees


ct = compare_trees.compare_trees

print(ct(tree1, tree2))

(1000, 997, 996, 90, 91, 0.09081786251881585)


## Defining an FP/FN Error Rate Function
The above tuple contains (number of leaves, number of edges in tree 1, number of edges in tree 2, FP, FN, RF distance). So to get the FP/FN rates we simply divide FN and FP by the number of edges in tree 1.

In [4]:
def fpn_rate(t):
    ''' Computes FP/FN rate given the tuple in the form above
    
    t:  tuple
        (nl, e1, e2, fp, fn, rf)
    '''
    return (t[3]/t[1], t[4]/t[1])

print(fpn_rate(ct(tree1, tree2)))

(0.09027081243731194, 0.09127382146439318)


## Calculating Error Rates
The following snippets will loop through each calculated tree and compute a dictionary containing average FP/FN rates across each of the five replicates (R0-R4) for each dataset and method.

In [5]:
err = {
    "1000M1": {
        "nj_logdet": [],
        "nj_jc": [],
        "nj_pdist": [],
        "ft_gtr": [],
        "ft_jc": []
    },
    "1000M4": {
        "nj_logdet": [],
        "nj_jc": [],
        "nj_pdist": [],
        "ft_gtr": [],
        "ft_jc": []
    }
}

In [6]:
from os import listdir



# Loop through each dataset and replicate
for f in listdir("data/"):
    for g in listdir(f"data/{f}"):
        # Load the true tree
        true_tree = dendropy.Tree.get(
            path=f"data/{f}/{g}/rose.tt", 
            schema="newick",
            rooting='force-unrooted',
            taxon_namespace=tns
        )
        
        # Load trees obtained from each method
        nj_logdet_tree = dendropy.Tree.get(
            path=f"data/{f}/{g}/nj_logdet/rose.aln.true.phylip_fastme_tree.txt", 
            schema="newick",
            rooting='force-unrooted',
            taxon_namespace=tns
        )
        
        nj_jc_tree = dendropy.Tree.get(
            path=f"data/{f}/{g}/nj_jc/rose.aln.true.phylip_fastme_tree.txt", 
            schema="newick",
            rooting='force-unrooted',
            taxon_namespace=tns
        )
        
        nj_pdist_tree = dendropy.Tree.get(
            path=f"data/{f}/{g}/nj_pdist/rose.aln.true.phylip_fastme_tree.txt", 
            schema="newick",
            rooting='force-unrooted',
            taxon_namespace=tns
        )
        
        ft_gtr_tree = dendropy.Tree.get(
            path=f"data/{f}/{g}/fasttree/gtrFastTree.tree", 
            schema="newick",
            rooting='force-unrooted',
            taxon_namespace=tns
        )
        
        ft_jc_tree = dendropy.Tree.get(
            path=f"data/{f}/{g}/fasttree/jcFastTree.tree", 
            schema="newick",
            rooting='force-unrooted',
            taxon_namespace=tns
        )
        
        # Gather bipartitions for FP/FN calculation
        true_tree.encode_bipartitions()
        nj_logdet_tree.encode_bipartitions()
        nj_jc_tree.encode_bipartitions()
        nj_pdist_tree.encode_bipartitions()
        ft_gtr_tree.encode_bipartitions()
        ft_jc_tree.encode_bipartitions()
        
        true_tree.collapse_basal_bifurcation(set_as_unrooted_tree=True)
        nj_logdet_tree.collapse_basal_bifurcation(set_as_unrooted_tree=True)
        nj_jc_tree.collapse_basal_bifurcation(set_as_unrooted_tree=True)
        nj_pdist_tree.collapse_basal_bifurcation(set_as_unrooted_tree=True)
        ft_gtr_tree.collapse_basal_bifurcation(set_as_unrooted_tree=True)
        ft_jc_tree.collapse_basal_bifurcation(set_as_unrooted_tree=True)
        
        err_nj_logdet = ct(true_tree, nj_logdet_tree)
        err_nj_jc = ct(true_tree, nj_jc_tree)
        err_nj_pdist = ct(true_tree, nj_pdist_tree)
        err_ft_gtr = ct(true_tree, ft_gtr_tree)
        err_ft_jc = ct(true_tree, ft_jc_tree)
        
        # Add results to arrays stored in error dictionary
        err[f]["nj_logdet"].append(fpn_rate(err_nj_logdet))
        err[f]["nj_jc"].append(fpn_rate(err_nj_jc))
        err[f]["nj_pdist"].append(fpn_rate(err_nj_pdist))
        err[f]["ft_gtr"].append(fpn_rate(err_ft_gtr))
        err[f]["ft_jc"].append(fpn_rate(err_ft_jc))

In [7]:
def pairwise_mean(a):
    ''' Return a tuple of columnwise averages
    
    a:  array
        Array of tuples
    '''
    fir = sum([e[0] for e in a])
    sec = sum([e[1] for e in a])
    
    return (round(fir/len(a), 3), round(sec/len(a), 3))

# Take columnwise averages of arrays in the error dictionary
for key in err:
    for subkey in err[key]:
        err[key][subkey] = pairwise_mean(err[key][subkey])

## Visualization
We will use Pandas to visualize the average (FP, FN) error rates for each dataset and method.

In [8]:
from pandas import DataFrame

DataFrame.from_dict(err)

,1000M1,1000M4
nj_logdet,"(0.213, 0.209)","(0.114, 0.084)"
nj_jc,"(0.226, 0.222)","(0.108, 0.078)"
nj_pdist,"(0.193, 0.189)","(0.127, 0.097)"
ft_gtr,"(0.11, 0.106)","(0.077, 0.047)"
ft_jc,"(0.129, 0.125)","(0.083, 0.053)"
